In [1]:
import cv2
import numpy as np
import imageio
import matplotlib.pyplot as plt

def CannyLines(frame, lower_bound, upper_bound, dilation=1):
    # canny edge detection on the grayscaled image
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    canny = cv2.Canny(blur, lower_bound, upper_bound)
    
    if dilation > 1: # if the line thickness we want is > 1 pixel
        kernel = np.ones((dilation, dilation), np.uint8) #also needs to be tuned
        img_dil = cv2.dilate(canny, kernel, iterations=1)
        return img_dil

    return canny

def MixedCannyLines(frame, ranges=[0,100,200,255], dilations = [1,2,4]):
    # grayscale and blur the images
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
  
    # template to which we keep adding the line images
    mixed_result = np.zeros_like(gray) 
    
    for idx, thickness in enumerate(dilations): # loop through all the line thicknesses we want

        canny = cv2.Canny(blur, ranges[idx], ranges[idx+1])  # 
        kernel = np.ones((thickness, thickness), np.uint8)  # make dilation kernel
        img_dil = cv2.dilate(canny, kernel, iterations=1)
        mixed_result = cv2.bitwise_or(mixed_result, img_dil)  # add the lines to the template image

    # reduce back to to 0-255 unit8 image
    mixed_result = np.clip(mixed_result, 0, 255)
    mixed_result = mixed_result.astype('uint8')
    
    return mixed_result



def blurImages(frame, prev_frame, blur=0.2):
    # convert frames to float values, to prevent numeric overflow with 8bit ints
    frame = np.asarray(frame, dtype=np.float32)
    prev_frame = np.asarray(prev_frame, dtype=np.float32) * blur
    
    blurred = frame + prev_frame

    # clip and convert back to 8bit int (so cv2 doesn't scream at me later)
    blurred = np.clip(blurred, 0, 255)
    blurred = blurred.astype('uint8')
    
    return blurred

In [2]:
# PARAMS
#framesize = (1080, 1920)  # desired output size
makeGIF = False
output_frames = []        # saves output
count = 0                # frame indxing
STEP_SIZE = 5          # how many frames to jump by (only opens every n-th frame)
black_frames = 10        # how many frames the fading out takes
black_counter = 0        # counter for that

# multi-line widths:
multiLine = True 
ranges=[0,100,200,255]   # first range is 0-100, then 100-200, etc.
dilations = [1,2,4]    # edges in range 0-100 are made 1 pixel wide, 100-200 is 2 pixels, etc...    more than 3 ranges should be possible
# if false, then just use these values (one constant line thickness):
lower_bound, upper_bound = 0, 60
dilation = 1

# blurring previous frames
blurring = True
blur = 0.2     # what % of the previously rendered frame to add to the current

# load video
cap = cv2.VideoCapture('Input Videos\Fulham_goal.mp4')
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# get first frame
ret, frame = cap.read()

if multiLine:
    prev_frame = MixedCannyLines(frame, ranges=ranges, dilations=dilations)
else:
    prev_frame = np.zeros_like(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY), dtype=np.uint8) 


while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        count += STEP_SIZE # skips 5 frames forward
        cap.set(cv2.CAP_PROP_POS_FRAMES, count)

        # make the edge detection lines
        if multiLine:
            canny_lines = MixedCannyLines(frame, ranges=ranges, dilations=dilations)
        else: # just one line thickness
            canny_lines = CannyLines(frame, lower_bound=lower_bound, upper_bound=upper_bound, dilation=dilation)

        # indexing, check to see if we want to fade out here
        remaining = length - 0
        if count > remaining: 
            decay_val = int((count - remaining) * 1.1)
            if decay_val > 255: 
                decay_val = 255
                black_counter += 1
                if black_counter > black_frames:
                    cap.release()
                    break 
            canny_lines = np.clip(canny_lines, 0+decay_val, 255.0)
            canny_lines -= decay_val 
        
        # blur some percent of previous frame into the current frame
        if blurring:
            result = blurImages(canny_lines, prev_frame, blur=blur)  
        else:
            result = canny_lines

        # show results, save output frame
        cv2.imshow("Frame", result)
        prev_frame = result
        if makeGIF:   output_frames.append(result)

    # for when we run out of frames  
    else:
        cap.release()
        break
    
    # to close the cv2 window, press the "q" key
    key = cv2.waitKey(5) & 0xFF
    if key == ord("q"):
        print("stopped at", count, "(video file) frames")
        break

cv2.destroyAllWindows()

# save the gif
if makeGIF:
    print("saving GIF...")
    with imageio.get_writer("Sing blur lines.gif", mode="I") as writer: # make a GIF output
        for idx, frame in enumerate(output_frames):
            #print("Adding frame to GIF file: ", idx + 1)
            writer.append_data(frame)

print("done!")
    

stopped at 100 (video file) frames
done!
